In [ ]:
import os
import sys
sys.path.append('/scratch_tmp/users/k21066795/BreastAgeNet')
import pandas as pd
from utils_features import *

In [ ]:
WSIs = '/scratch_tmp/prj/cb_normalbreast/prj_BreastAgeNet/WSIs'
FEATURES = '/scratch_tmp/prj/cb_normalbreast/prj_BreastAgeNet/FEATURES'
model_name = 'UNI'
batch_size = 16
num_workers = 2
stainFunc = 'augmentation'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"device: {device}", flush=True)
model, transform = get_model(model_name, device)
print(f"Model: {model_name}", flush=True)

In [ ]:
# extract features from WSIs (e.g., ".ndpi" files)
wsinames = os.listdir(f"{FEATURES}/KHP_RM")
for wsi_id in wsinames:
    output_dir = f"{FEATURES}/KHP_RM/{wsi_id}"
    fname = f"{output_dir}/{wsi_id}_bagFeature_{model_name}_{stainFunc}.h5"
    if not os.path.exists(fname):
        print(wsi_id)
        file = glob.glob(f"{FEATURES}/*/{wsi_id}*/{wsi_id}*_patch*.csv")
        if file:
            print(file[0])
            bag_df = pd.read_csv(file[0])
            print(len(bag_df))
            if len(bag_df)>0:
                bag_dataset = Dataset_fromWSI(bag_df, WSIs, stainFunc, transforms_eval=transform)
                extract_features(model, bag_dataset, batch_size, num_workers, device, fname)

In [ ]:
# extract features from Patches (e.g., ".png" files)
pt = f"/scratch_tmp/prj/cb_normalbreast/prj_NBTClassifier/TC512_externaltesting_EPFL.csv"
df = pd.read_csv(pt)
df["wsi_id"] = [os.path.basename(i).split("_HE")[0] for i in list(df['file_path'])]
df["patch_id"] = [os.path.basename(i).split(".png")[0] for i in list(df['file_path'])]
print(len(df))
# df = df.loc[df['TC_epi'] > 0.9, :].copy()
# print(len(df))
patch_df = df.copy()
output_dir = "/scratch_tmp/prj/cb_normalbreast/prj_BreastAgeNet/FEATURES/EPFL"
for wsi_id, bag_df in patch_df.groupby('wsi_id'):
    print(f"Processing WSI: {wsi_id}", flush=True)
    bag_dataset = Dataset_frompatch(bag_df, stainFunc, transform)
    fname = f"{output_dir}/{wsi_id}_bagFeature_{model_name}_{stainFunc}_embeddings.h5"
    extract_features(model, bag_dataset, batch_size, num_workers, device, fname)